In [1]:
import casbin

In [2]:
with open("./tests/model.conf", "w") as conf_file:
    conf_file.write(
"""
[request_definition]
r = sub, obj, act

[policy_definition]
p = sub, obj, act

[policy_effect]
e = some(where (p.eft == allow))

[matchers]
m = r.sub == p.sub && r.obj == p.obj && r.act == p.act
""")

In [6]:
with open("./tests/policy.csv", "r") as policy_file:
    print(policy_file.read())

p, ark:99999/test-user, canCreateProject, ark:99999/CAMA
p, ark:99999/test-user, delete, ark:99999/CAMA-HCTSA
p, ark:99999/test-user, update, ark:99999/CAMA-HCTSA
p, ark:99999/test-user, managePermissions, ark:99999/CAMA-HCTSA


In [3]:
enforcer = casbin.Enforcer("./tests/model.conf", "./tests/policy.csv")
enforcer.load_policy()

## Flow to Test
1. our test user is signs up
2. that user is granted access to an organization
3. that user creates a new project within that organization
4. one of each object is created within that new project
5. that user creates a new group of other users
6. the test user gives access to the project for the group they created

### Permissions

#### Organization
- can create project
- can edit organization metadata
- can delete projects

#### Project
- can add a digital object
- can delete the entire project
- can edit project metadata
- can delete any digital object within the project
- can change project permissions

In [8]:
user = "ark:99999/test-user"
org = "ark:99999/CAMA"
proj = "ark:99999/CAMA-HCTSA"

dataset = "ark:99999/CAMA-HCTSA-test-dataset"
dataset_object = "ark:99999/CAMA-HCTSA-test-dataset-download"

software = "ark:99999/CAMA-HCTSA-test-software"
software_object = "ark:99999/CAMA-HCTSA-test-software-download"

computation = "ark:99999/CAMA-HCTSA-computation"

result = "ark:99999/CAMA-HCTSA-computation-result"
result_object = "ark:99999/CAMA-HCTSA-computation-result-download"

In [24]:
# step 1 a user is granted access to an organization
# insert a policy
enforcer.add_policy(user, "canCreateProject", org)

# persisting the policies to storage
enforcer.save_policy()

In [9]:
# test that the user can actually create the project
enforcer.enforce(user, "canCreateProject", org)

True

In [28]:
# a user creates a project
enforcer.add_policies([
    (user, "delete", proj), 
    (user, "update", proj), 
    (user, "managePermissions", proj),
    (user, "createDigitalObject", proj)])

enforcer.save_policy()